# Segmenting and Clustering Neighborhoods in Toronto

## Part One - Creating the dataframe

Importing libraries

In [1]:
import pandas as pd
import numpy as np

* Reading the given wikipedia page URL to a pandas dataframe
* Looking for the table containing "Postal Code"s
* Setting the first row to headers
* Setting "Not assigned" to be considered missing values so we can use the dropna function

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',
             match= 'Postal Code', header=0, na_values='Not assigned')[0]

Drop "Not assigned" boroughs

In [3]:
df.dropna(subset=['Borough'], inplace=True)

Making sure there are no postal codes with a borough but a not assigned neighborhood

In [4]:
df[df['Neighborhood'].isna()]

Empty DataFrame
Columns: [Postal Code, Borough, Neighborhood]
Index: []

Reset the index and view the dataframe

In [5]:
df.reset_index(drop=True, inplace=True)
df

Postal Code           Borough  \
0           M3A        North York   
1           M4A        North York   
2           M5A  Downtown Toronto   
3           M6A        North York   
4           M7A  Downtown Toronto   
5           M9A         Etobicoke   
6           M1B       Scarborough   
7           M3B        North York   
8           M4B         East York   
9           M5B  Downtown Toronto   
10          M6B        North York   
11          M9B         Etobicoke   
12          M1C       Scarborough   
13          M3C        North York   
14          M4C         East York   
15          M5C  Downtown Toronto   
16          M6C              York   
17          M9C         Etobicoke   
18          M1E       Scarborough   
19          M4E      East Toronto   
20          M5E  Downtown Toronto   
21          M6E              York   
22          M1G       Scarborough   
23          M4G         East York   
24          M5G  Downtown Toronto   
25          M6G  Downtown Toronto   
26          M1H       Scarborough   
27          M2H        North York   
28          M3H        North York   
29          M4H         East York   
..          ...               ...   
73          M4R   Central Toronto   
74          M5R   Central Toronto   
75          M6R      West Toronto   
76          M7R       Mississauga   
77          M9R         Etobicoke   
78          M1S       Scarborough   
79          M4S   Central Toronto   
80          M5S  Downtown Toronto   
81          M6S      West Toronto   
82          M1T       Scarborough   
83          M4T   Central Toronto   
84          M5T  Downtown Toronto   
85          M1V       Scarborough   
86          M4V   Central Toronto   
87          M5V  Downtown Toronto   
88          M8V         Etobicoke   
89          M9V         Etobicoke   
90          M1W       Scarborough   
91          M4W  Downtown Toronto   
92          M5W  Downtown Toronto   
93          M8W         Etobicoke   
94          M9W         Etobicoke   
95          M1X       Scarborough   
96          M4X  Downtown Toronto   
97          M5X  Downtown Toronto   
98          M8X         Etobicoke   
99          M4Y  Downtown Toronto   
100         M7Y      East Toronto   
101         M8Y         Etobicoke   
102         M8Z         Etobicoke   

                                          Neighborhood  
0                                            Parkwoods  
1                                     Victoria Village  
2                            Regent Park, Harbourfront  
3                     Lawrence Manor, Lawrence Heights  
4          Queen's Park, Ontario Provincial Government  
5              Islington Avenue, Humber Valley Village  
6                                       Malvern, Rouge  
7                                            Don Mills  
8                      Parkview Hill, Woodbine Gardens  
9                             Garden District, Ryerson  
10                                           Glencairn  
11   West Deane Park, Princess Gardens, Martin Grov...  
12              Rouge Hill, Port Union, Highland Creek  
13                                           Don Mills  
14                                    Woodbine Heights  
15                                      St. James Town  
16                                  Humewood-Cedarvale  
17   Eringate, Bloordale Gardens, Old Burnhamthorpe...  
18                   Guildwood, Morningside, West Hill  
19                                         The Beaches  
20                                         Berczy Park  
21                                 Caledonia-Fairbanks  
22                                              Woburn  
23                                             Leaside  
24                                  Central Bay Street  
25                                            Christie  
26                                           Cedarbrae  
27                                   Hillcrest Village  
28     Bathurst Manor, Wilson Heights, Downsview North  

Using the .shape method to print the number of rows of the dataframe

In [6]:
df.shape

(103, 3)

## Part Two - Get Latitude and Longitude

Installing & importing the Geocoder package

In [7]:
import geocoder

Tried to use the geocoder package to get the latitudes & longitudes for all the postal codes but it was taking forever

In [8]:
lats, longs = [], []
codes = df['Postal Code']
for code in codes:
    coords = None
    while coords == None:
        g = geocoder.google('{}, Toronto, Ontario'.format(code))
        coords = g.latlng
    lats.append(coords[0])
    longs.append(coords[1])

KeyboardInterrupt: 

Then tried to just get the coordinates for a single postal code and limited it to 100 tries

In [9]:
# initialize your variable to None
lat_lng_coords = None
postal_code = 'M5G'
tries = 0

# loop until you get the coordinates
while tries < 100:
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng
    tries += 1
    if lat_lng_coords:
        break

if lat_lng_coords:
    print('got em')
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
else:
    print('no go')

no go


Still not working - got the coordiantes from the csv file

In [10]:
coords = pd.read_csv('Geospatial_Coordinates.csv')
coords.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

Merging the original dataframe with the new dataframe containing latitudes & longitudes

In [11]:
TorontoMetro = df.join(coords.set_index('Postal Code'), on='Postal Code')
TorontoMetro.head()

Postal Code           Borough                                 Neighborhood  \
0         M3A        North York                                    Parkwoods   
1         M4A        North York                             Victoria Village   
2         M5A  Downtown Toronto                    Regent Park, Harbourfront   
3         M6A        North York             Lawrence Manor, Lawrence Heights   
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494

## Part Three - Explore and Cluster

Importing Stuff

In [12]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [13]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [41]:
import requests # library to handle requests

In [14]:
import folium # map rendering library

In [15]:
Tlat, Tlong = 43.6532, -79.3832

Create a map of Tornto and its neighborhoods

In [17]:
# create map of Toronto
TorontoMetroMap = folium.Map(location=[Tlat, Tlong], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(TorontoMetro['Latitude'], 
                                           TorontoMetro['Longitude'], 
                                           TorontoMetro['Borough'], 
                                           TorontoMetro['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(TorontoMetroMap)  
    
TorontoMetroMap

Check how many neighborhoods are in each borough

In [18]:
TorontoMetro.groupby(['Borough']).count()

Postal Code  Neighborhood  Latitude  Longitude
Borough                                                         
Central Toronto             9             9         9          9
Downtown Toronto           19            19        19         19
East Toronto                5             5         5          5
East York                   5             5         5          5
Etobicoke                  12            12        12         12
Mississauga                 1             1         1          1
North York                 24            24        24         24
Scarborough                17            17        17         17
West Toronto                6             6         6          6
York                        5             5         5          5

Create a dataframe for just neighborhoods in Toronto proper (boroughs named Toronto) 

In [19]:
TorontoProper = TorontoMetro[TorontoMetro['Borough'].str.contains('Toronto')]
TorontoProper.reset_index(drop=True, inplace=True)
TorontoProper.head()

Postal Code           Borough                                 Neighborhood  \
0         M5A  Downtown Toronto                    Regent Park, Harbourfront   
1         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
2         M5B  Downtown Toronto                     Garden District, Ryerson   
3         M5C  Downtown Toronto                               St. James Town   
4         M4E      East Toronto                                  The Beaches   

    Latitude  Longitude  
0  43.654260 -79.360636  
1  43.662301 -79.389494  
2  43.657162 -79.378937  
3  43.651494 -79.375418  
4  43.676357 -79.293031

Check how many neighborhoods are in Toronto proper & list them

In [20]:
TorontoProper.shape

(39, 5)

In [21]:
for hood in TorontoProper['Neighborhood']:
    print(hood)

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

Get rid of the PO Boxes & Business reply neighborhoods

In [32]:
pobox = TorontoProper[TorontoProper['Neighborhood'].str.contains('PO Boxes')]
reply = TorontoProper[TorontoProper['Neighborhood'].str.contains('Business reply')]
TorontoProper.drop(index=[pobox.index[0], reply.index[0]], inplace=True)
TorontoProper.reset_index(drop=True, inplace=True)
TorontoProper

C:\Users\Corinne\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Postal Code           Borough  \
0          M5A  Downtown Toronto   
1          M7A  Downtown Toronto   
2          M5B  Downtown Toronto   
3          M5C  Downtown Toronto   
4          M4E      East Toronto   
5          M5E  Downtown Toronto   
6          M5G  Downtown Toronto   
7          M6G  Downtown Toronto   
8          M5H  Downtown Toronto   
9          M6H      West Toronto   
10         M5J  Downtown Toronto   
11         M6J      West Toronto   
12         M4K      East Toronto   
13         M5K  Downtown Toronto   
14         M6K      West Toronto   
15         M4L      East Toronto   
16         M5L  Downtown Toronto   
17         M4M      East Toronto   
18         M4N   Central Toronto   
19         M5N   Central Toronto   
20         M4P   Central Toronto   
21         M5P   Central Toronto   
22         M6P      West Toronto   
23         M4R   Central Toronto   
24         M5R   Central Toronto   
25         M6R      West Toronto   
26         M4S   Central Toronto   
27         M5S  Downtown Toronto   
28         M6S      West Toronto   
29         M4T   Central Toronto   
30         M5T  Downtown Toronto   
31         M4V   Central Toronto   
32         M5V  Downtown Toronto   
33         M4W  Downtown Toronto   
34         M4X  Downtown Toronto   
35         M5X  Downtown Toronto   
36         M4Y  Downtown Toronto   

                                         Neighborhood   Latitude  Longitude  
0                           Regent Park, Harbourfront  43.654260 -79.360636  
1         Queen's Park, Ontario Provincial Government  43.662301 -79.389494  
2                            Garden District, Ryerson  43.657162 -79.378937  
3                                      St. James Town  43.651494 -79.375418  
4                                         The Beaches  43.676357 -79.293031  
5                                         Berczy Park  43.644771 -79.373306  
6                                  Central Bay Street  43.657952 -79.387383  
7                                            Christie  43.669542 -79.422564  
8                            Richmond, Adelaide, King  43.650571 -79.384568  
9                        Dufferin, Dovercourt Village  43.669005 -79.442259  
10  Harbourfront East, Union Station, Toronto Islands  43.640816 -79.381752  
11                           Little Portugal, Trinity  43.647927 -79.419750  
12                       The Danforth West, Riverdale  43.679557 -79.352188  
13           Toronto Dominion Centre, Design Exchange  43.647177 -79.381576  
14       Brockton, Parkdale Village, Exhibition Place  43.636847 -79.428191  
15                     India Bazaar, The Beaches West  43.668999 -79.315572  
16                     Commerce Court, Victoria Hotel  43.648198 -79.379817  
17                                    Studio District  43.659526 -79.340923  
18                                      Lawrence Park  43.728020 -79.388790  
19                                           Roselawn  43.711695 -79.416936  
20                                   Davisville North  43.712751 -79.390197  
21    Forest Hill North & West, Forest Hill Road Park  43.696948 -79.411307  
22                      High Park, The Junction South  43.661608 -79.464763  
23                  North Toronto West, Lawrence Park  43.715383 -79.405678  
24                The Annex, North Midtown, Yorkville  43.672710 -79.405678  
25                             Parkdale, Roncesvalles  43.648960 -79.456325  
26                                         Davisville  43.704324 -79.388790  
27                     University of Toronto, Harbord  43.662696 -79.400049  
28                                 Runnymede, Swansea  43.651571 -79.484450  
29                        Moore Park, Summerhill East  43.689574 -79.383160  
30          Kensington Market, Chinatown, Grange Park  43.653206 -79.400049  
31  Summerhill West, Rathnelly, South Hill, Forest...  43.686412 -79.400049  
32  CN Tower, King and Spadina, Railway Lands, Har...  43.62

Plot all of the neighborhoods in Toronto proper on a map

In [35]:
# create map of Toronto Proper
TorontoProperMap = folium.Map(location=[Tlat, Tlong], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(TorontoProper['Latitude'], 
                                           TorontoProper['Longitude'], 
                                           TorontoProper['Borough'], 
                                           TorontoProper['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(TorontoProperMap)  
    
TorontoProperMap

Define function to get the venues in each neighborhood

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):  
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Call the function to get all the venues

In [42]:
toronto_venues = getNearbyVenues(names=TorontoProper['Neighborhood'],
                                   latitudes=TorontoProper['Latitude'],
                                   longitudes=TorontoProper['Longitude'])

In [43]:
toronto_venues.head()

Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Regent Park, Harbourfront               43.65426              -79.360636   
1  Regent Park, Harbourfront               43.65426              -79.360636   
2  Regent Park, Harbourfront               43.65426              -79.360636   
3  Regent Park, Harbourfront               43.65426              -79.360636   
4  Regent Park, Harbourfront               43.65426              -79.360636   

                    Venue  Venue Latitude  Venue Longitude  \
0        Roselle Desserts       43.653447       -79.362017   
1           Tandem Coffee       43.653559       -79.361809   
2      Morning Glory Cafe       43.653947       -79.361149   
3  Cooper Koo Family YMCA       43.653249       -79.358008   
4     Body Blitz Spa East       43.654735       -79.359874   

        Venue Category  
0               Bakery  
1          Coffee Shop  
2       Breakfast Spot  
3  Distribution Center  
4                  Spa

In [45]:
print(toronto_venues.shape)
toronto_venues.groupby('Neighborhood').count()

(1501, 7)


Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            54   
Brockton, Parkdale Village, Exhibition Place                           23   
CN Tower, King and Spadina, Railway Lands, Harb...                     16   
Central Bay Street                                                     63   
Christie                                                               17   
Church and Wellesley                                                   77   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             34   
Davisville North                                                        8   
Dufferin, Dovercourt Village                                           16   
First Canadian Place, Underground city                                100   
Forest Hill North & West, Forest Hill Road Park                         4   
Garden District, Ryerson                                              100   
Harbourfront East, Union Station, Toronto Islands                     100   
High Park, The Junction South                                          25   
India Bazaar, The Beaches West                                         22   
Kensington Market, Chinatown, Grange Park                              60   
Lawrence Park                                                           3   
Little Portugal, Trinity                                               45   
Moore Park, Summerhill East                                             4   
North Toronto West, Lawrence Park                                      22   
Parkdale, Roncesvalles                                                 14   
Queen's Park, Ontario Provincial Government                            34   
Regent Park, Harbourfront                                              44   
Richmond, Adelaide, King                                               94   
Rosedale                                                                4   
Roselawn                                                                4   
Runnymede, Swansea                                                     34   
St. James Town                                                         79   
St. James Town, Cabbagetown                                            43   
Studio District                                                        40   
Summerhill West, Rathnelly, South Hill, Forest ...                     16   
The Annex, North Midtown, Yorkville                                    21   
The Beaches                                                             4   
The Danforth West, Riverdale                                           43   
Toronto Dominion Centre, Design Exchange                              100   
University of Toronto, Harbord                                         34   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             54   
Brockton, Parkdale Village, Exhibition Place                            23   
CN Tower, King and Spadina, Railway Lands, Harb...                      16   
Central Bay Street                                                      63   
Christie                                                                17   
Church and Wellesley                                                    77   
Commerce Court, Victoria Hotel                                         100   
Davisville                                                              34   
Davisville North                                                         8   
Dufferin, Dovercourt Village                                            16   
First Canadian Place, Underground city                                 100   
Forest Hill North & West, Forest H

One-hot encode then group by neighborhood

In [60]:
TorontoHot = pd.get_dummies(toronto_venues['Venue Category'])
TorontoHot['Neighborhood'] = toronto_venues['Neighborhood']
TorontoGrouped = TorontoHot.groupby('Neighborhood').mean().reset_index()
TorontoGrouped.head()

Neighborhood  Afghan Restaurant  \
0                                        Berczy Park                0.0   
1       Brockton, Parkdale Village, Exhibition Place                0.0   
2  CN Tower, King and Spadina, Railway Lands, Har...                0.0   
3                                 Central Bay Street                0.0   
4                                           Christie                0.0   

   Airport  Airport Food Court  Airport Lounge  Airport Service  \
0   0.0000              0.0000           0.000           0.0000   
1   0.0000              0.0000           0.000           0.0000   
2   0.0625              0.0625           0.125           0.1875   
3   0.0000              0.0000           0.000           0.0000   
4   0.0000              0.0000           0.000           0.0000   

   Airport Terminal  American Restaurant  Antique Shop  Aquarium  ...  Trail  \
0             0.000                  0.0           0.0       0.0  ...    0.0   
1             0.000                  0.0           0.0       0.0  ...    0.0   
2             0.125                  0.0           0.0       0.0  ...    0.0   
3             0.000                  0.0           0.0       0.0  ...    0.0   
4             0.000                  0.0           0.0       0.0  ...    0.0   

   Train Station  Vegetarian / Vegan Restaurant  Video Game Store  \
0            0.0                       0.018519               0.0   
1            0.0                       0.000000               0.0   
2            0.0                       0.000000               0.0   
3            0.0                       0.015873               0.0   
4            0.0                       0.000000               0.0   

   Vietnamese Restaurant  Wine Bar  Wine Shop  Wings Joint  Women's Store  \
0                    0.0  0.000000        0.0          0.0            0.0   
1                    0.0  0.000000        0.0          0.0            0.0   
2                    0.0  0.000000        0.0          0.0            0.0   
3                    0.0  0.015873        0.0          0.0            0.0   
4                    0.0  0.000000        0.0          0.0            0.0   

   Yoga Studio  
0     0.000000  
1     0.000000  
2     0.000000  
3     0.015873  
4     0.000000  

[5 rows x 228 columns]

### Run k-means to get clusters

In [116]:
clusters = 6
TorontoClustered = TorontoGrouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters = i, random_state=0)
kmeans.fit(TorontoClustered)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=8, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

Merge clusters with Neighborhoods

In [97]:
TorontoProper['Cluster'] = kmeans.labels_
TorontoProper.head()

C:\Users\Corinne\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Postal Code           Borough                                 Neighborhood  \
0         M5A  Downtown Toronto                    Regent Park, Harbourfront   
1         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
2         M5B  Downtown Toronto                     Garden District, Ryerson   
3         M5C  Downtown Toronto                               St. James Town   
4         M4E      East Toronto                                  The Beaches   

    Latitude  Longitude  Cluster  
0  43.654260 -79.360636        5  
1  43.662301 -79.389494        5  
2  43.657162 -79.378937        5  
3  43.651494 -79.375418        0  
4  43.676357 -79.293031        5

See how many neighborhoods are in each cluster

In [117]:
TorontoProper.groupby('Cluster').count()

Postal Code  Borough  Neighborhood  Latitude  Longitude
Cluster                                                         
0                 14       14            14        14         14
1                  2        2             2         2          2
2                  1        1             1         1          1
3                  1        1             1         1          1
4                  1        1             1         1          1
5                 18       18            18        18         18

Visualize the different clusters on a map

In [118]:
# create map
map_clusters = folium.Map(location=[Tlat, Tlong], zoom_start=11)

# set color scheme for the clusters
x = np.arange(clusters)
ys = [i + x + (i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(TorontoProper['Latitude'], 
                                  TorontoProper['Longitude'], 
                                  TorontoProper['Neighborhood'], 
                                  TorontoProper['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters